In [1]:
import Text.Megaparsec
import Text.Megaparsec.Char
import Data.Char
import qualified Data.Text as T
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Sampler
import Control.Monad.Bayes.Weighted
import Control.Monad.Bayes.Population
import Control.Monad.Bayes.Inference.SMC
import Control.Monad.Trans (lift)
import Control.Monad (join, replicateM)
import Data.Void
import Text.Pretty.Simple
:e OverloadedStrings
:e FlexibleContexts
:e GADTs
:e LambdaCase

In [96]:
noise = \case
    'g' -> lift $ uniformD ['g','h']
    'h' -> return 'h'

parser = do
    true <- lift $ uniformD ['g', 'h']
    predicted <- noise true
    observed <- lookAhead (char 'g' <|> char 'h')
    lift $ condition $ predicted == observed
    char observed
    return true 

errorBundlePretty' :: (TraversableStream s, VisualStream s) => ParseErrorBundle s Void -> String 
errorBundlePretty' = errorBundlePretty

-- foo :: MonadInfer m => m T.Text

run parser input = (either (T.pack . errorBundlePretty' ) (T.pack . show) ) <$> runParserT parser "" input

Line 18: Redundant bracket
Found:
(either (T.pack . errorBundlePretty') (T.pack . show))
  <$> runParserT parser "" input
Why not:
either (T.pack . errorBundlePretty') (T.pack . show)
  <$> runParserT parser "" input

In [97]:
x <- sampleIO $ runPopulation $ smcMultinomial 2 100 $ run (some parser <* eof) "hh"
pPrint $ toEmpiricalWeighted x 

[
    ( ""hh""
    , 0.53
    )
,
    ( ""hg""
    , 0.23
    )
,
    ( ""gg""
    , 0.1399999999999999
    )
,
    ( ""gh""
    , 9.999999999999998 e- 2
    )
]

In [74]:
import Debug.Trace

two = do
    x <- parser
    y <- parser
    lift $ factor $ case (x,y) of ('h','g') -> 1; _ -> 0.5
    return [x ,y]
    
duplicate = do
    -- x <- parser
    b <- lift $ bernoulli 0.5
    if b then "h" else "g"
    -- char x
    -- lift $ factor $ case (x,y) of ('h','g') -> 1; _ -> 0.5
    -- return x
    


In [75]:
import Control.Monad.Bayes.Enumerator

x <- sampleIO $ runPopulation $ smcMultinomial 2 100 $ run (duplicate) "hg"
pPrint $ toEmpiricalWeighted x

[
    ( ""h""
    , 0.54
    )
,
    ( "1:1:
        |
      1 | hg
        | ^
      unexpected 'h'
      expecting 'g'
      "
    , 0.45999999999999996
    )
]